In [11]:
import pandas as pd
import numpy as np
from collections import Counter
import re

from sklearn import preprocessing

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

import seaborn as sns
sns.set()
%matplotlib inline
import string

import os 
from datetime import datetime
import random
import math
from sklearn.cross_validation import train_test_split


In [2]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;


from xgboost import XGBClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [3]:
def preprocess(df):
    # reassign id
#     df.index = df.PassengerId

    # drop not imp data
#     df.drop("PassengerId", axis=1, inplace=True)
    
    # avg_age
#     avg_age = df['Age'].mean()
    avg_age = df['Age'].median()
#     std = df['Age'].std()
#     mean = df['Age'].mean()
#     size = len(df[pd.isnull(df['Age'])])
#     age_null_random_list = np.random.randint(mean - std, mean + std, size=size)
#     df.loc[pd.isnull(df['Age']), 'Age'] = age_null_random_list
    
    # age
#     age_range = list(set(pd.qcut(df[pd.notnull(df['Age'])]['Age'], 5)))
#     def agecat(age):
#         if age in age_range[0]:
#             cat = 0
#         elif age in age_range[1]:
#             cat = 1
#         elif age in age_range[2]:
#             cat = 2
#         elif age in age_range[3]:
#             cat = 3
#         else:
#             cat = 4
#         return cat
    
    def agecat(age):
        if age <= 16:
            cat = 0
        elif age > 16 and age <= 32:
            cat = 1
        elif age > 32 and age <= 48:
            cat = 2
        elif age < 48 and age <= 64:
            cat = 3
        else:
            cat = 4
        return cat
    
    
    # fare
#     fare_range = list(set(pd.qcut(df[pd.notnull(df['Fare'])]['Fare'], 7)))
#     def farecat(f):
#         if f in fare_range[0]:
#             cat = 0
#         elif f in fare_range[1]:
#             cat = 1
#         elif f in fare_range[2]:
#             cat = 2
#         elif f in fare_range[3]:
#             cat = 3
#         elif f in fare_range[4]:
#             cat = 4
#         elif f in fare_range[5]:
#             cat = 5
#         else:
#             cat = 6
#         return cat
    
    
    def farecat(f):
        if f <= 16:
            cat = 0
        elif f > 17 and f <= 32:
            cat = 1
        elif f > 32 and f <= 48:
            cat = 2
        elif f < 48 and f <= 64:
            cat = 3
        elif f < 64 and f <= 80:
            cat = 4
        elif f < 80 and f <= 96:
            cat = 5
        else:
            cat = 6
        return cat

    
    # avg_fare
    fares_notnull = df[pd.notnull(df['Fare'])]['Fare']
    avg_fare = fares_notnull.median()

    # ticket
    ticket_cat = {}
    for ticket in df['Ticket']:
        if ticket.isdigit():
            ticket_cat[ticket] = 1
        elif ticket.startswith('A'):
            ticket_cat[ticket] = 2
        elif ticket.startswith('C'):
            ticket_cat[ticket] = 3
        elif ticket.startswith('F'):
            ticket_cat[ticket] = 4
        elif ticket.startswith('P'):
            ticket_cat[ticket] = 5
        elif ticket.startswith('SOTON'):
            ticket_cat[ticket] = 6
        elif ticket.startswith('STON'):
            ticket_cat[ticket] = 7
        elif ticket.startswith('S'):
            ticket_cat[ticket] = 8
        elif ticket.startswith('W'):
            ticket_cat[ticket] = 9
        else:
            ticket_cat[ticket] = 0
            
    ticket_cat1 = {}
    for num, name in enumerate(list(set([item.split()[0].replace(".", "").replace("/", "") for item in df['Ticket'] if not item.isdigit()]))):
        ticket_cat1[name] = num
    
    # cabin
    cabin_cat = {}
    for cabin in df['Cabin']:
        if pd.isnull(cabin):
            cabin_cat[cabin] = 0
        elif cabin.startswith('A'):
            cabin_cat[cabin] = 1
        elif cabin.startswith('B'):
            cabin_cat[cabin] = 2
        elif cabin.startswith('C'):
            cabin_cat[cabin] = 3
        elif cabin.startswith('D'):
            cabin_cat[cabin] = 4
        elif cabin.startswith('E'):
            cabin_cat[cabin] = 5
        else:
            cabin_cat[cabin] = 0
    
    # embarked
    embarked_cat = {}
    for embarked in df['Embarked']:
        if pd.isnull(embarked):
            embarked_cat[embarked] = 0
        elif embarked.startswith('S'):
            embarked_cat[embarked] = 0
        elif embarked.startswith('Q'):
            embarked_cat[embarked] = 1
        elif embarked.startswith('C'):
            embarked_cat[embarked] = 2
            
            
    # title
    title_mapping= {
        'Ms':"Miss",
        'Mlle':"Miss",
        'Miss':"Miss",
        'Mrs':"Mrs",
        'Mme':"Mrs",
        'MrsMartin(ElizabethL':"Mrs",
        'Mr':"Mr"
        }
    
    title_cat = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    
    # Name_with specail chars
    def withspecailchar(name):
        for char in name:
            if char in string.punctuation:
                return 1
        return 0
    
    def applyfun(row):
        row['Has_Cabin'] = 0 if pd.isnull(row['Cabin']) else 1
        row['Cabin'] = cabin_cat.get(row['Cabin'])

        row['Age'] = row['Age'] if pd.notnull(row['Age']) else avg_age
        row['Age_Cat'] = agecat(row['Age'])
        row['Age_Lower_than_15'] = 1 if row['Age'] < 15 else 0
        row['Age_Higher_than_75'] = 1 if row['Age'] > 75 else 0

        row['Fare'] = row['Fare'] if pd.notnull(row['Fare']) else avg_fare
        row['Fare_log10'] = math.log(row['Fare'], 10) if  row['Fare'] != 0 else 0
        row['Fare_log2'] = math.log(row['Fare'], 2) if  row['Fare'] != 0 else 0

        row['Fare_Cat'] = farecat(row['Fare'])
        row['High_Price_Fare'] = 1 if row['Fare'] > 200 else 0

        row['Ticket'] = ticket_cat.get(row['Ticket'])
        processed_ticket = row['Ticket'].split()[0].replace(".", "").replace("/", "") if not str(row['Ticket']).isdigit() else None
#         row['Ticket1'] = ticket_cat1.get(processed_ticket) if processed_ticket != None else 30

        row['Embarked'] = embarked_cat.get(row['Embarked'])
        row['Sex'] = 1 if row['Sex'] == 'male' else 0

        row['Name_Length'] = len(row['Name'])
        row['Name_With_Special_Char'] = withspecailchar(row['Name'].replace(',', "").replace('.', ""))
        row['Family_Size'] = row['SibSp'] + row['Parch']
        row['Is_Alone']= 1 if row['Family_Size'] == 1 else 0
        
        call = re.search(r'\,.+\.', row['Name']).group(0).replace(",", "").replace(".", "").replace(" ", "")
        call_cat = title_cat.get(title_mapping.get(call, "Rare"))
        row['Title'] = call_cat
#         row['Mother'] = 1 if row['Age'] >18 and row['Parch'] > 0 and call != 'Miss' else 0
        return row
    
    df = df.apply(applyfun, axis=1)
    df.drop('Name', axis=1, inplace=True)
    df.drop('Fare', axis=1, inplace=True)


    return df

In [4]:
# Input Data Preparation
df = pd.read_csv('train.csv')
df = preprocess(df)
x_train = np.matrix(df.drop(['Survived','PassengerId'], axis=1))
y_train = np.array(df['Survived'])

df_test = pd.read_csv('test.csv')
ID = df_test['PassengerId']
x_test = np.matrix(preprocess(df_test.drop('PassengerId', axis=1)))

print(y_train.shape)
print(x_train.shape)
print(x_test.shape)

(891,)
(891, 21)
(418, 21)


In [5]:
# Some useful parameters which will come in handy later on
ntrain = df.shape[0]
ntest = df_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        if seed == 0:
            params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [13]:
def get_oof_tuning(clf, x_train, y_train, x_test):
    x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(x_train, y_train, test_size=0.4, random_state=4242)
    clf.train(x_train_split, y_train_split)
    x_valid_predict = clf.predict(x_valid_split)  
    accuracy1 = np.sum(x_valid_predict == y_valid_split)/len(x_valid_predict)
    
    x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(x_train, y_train, test_size=0.4, random_state=1111)
    clf.train(x_train_split, y_train_split)
    x_valid_predict = clf.predict(x_valid_split)  
    accuracy2 = np.sum(x_valid_predict == y_valid_split)/len(x_valid_predict)

    return (accuracy1 + accuracy2) / 2


# param tuning
def gen_params(input_params):
    test_params = []
    for i in range(1000):
        param = {}
        for key, values in input_params.items():
            param[key] = np.random.choice(values)
        test_params.append(param)

    df_test_params = pd.DataFrame(test_params)
    df_test_params = df_test_params.drop_duplicates()
    test_params = df_test_params.T.to_dict()
    return [value for key, value in test_params.items()]


def find_best_params(test_params, clf_type):
    best_score = 0
    best_param = {}
    for num, param in enumerate(test_params):
        clf = SklearnHelper(clf=clf_type, seed=SEED, params=param)
        accuracy = get_oof_tuning(clf, x_train, y_train, x_test) # Random Forest
        if accuracy > best_score:
            best_score = accuracy
            best_param = param
            print("number:", num)
            print("params:", best_param)
            print("accuracy:", best_score)
    return best_score, best_param

In [7]:
# rf_bestscore = [{'params': {'criterion': 'gini',
#    'max_depth': 28,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 335,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0,
#    'warm_start': True},
#   'score': 0.9079685746352413},
#  {'params': {'criterion': 'gini',
#    'max_depth': 28,
#    'max_features': None,
#    'min_samples_leaf': 2,
#    'n_estimators': 337,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0,
#    'warm_start': True},
#   'score': 0.92368125701459036},
#  {'params': {'criterion': 'gini',
#    'max_depth': 25,
#    'max_features': None,
#    'min_samples_leaf': 2,
#    'n_estimators': 345,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0,
#    'warm_start': True},
#   'score': 0.92480359147025815}]



# et_bestscore = [{'params': {'max_depth': 20,
#    'max_features': 'log2',
#    'min_samples_leaf': 2,
#    'n_estimators': 160,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.82940516273849607},
#  {'params': {'max_depth': 20,
#    'max_features': 'log2',
#    'min_samples_leaf': 2,
#    'n_estimators': 162,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.83052749719416386}]



# ada_bestscore = [{'params': {'algorithm': 'SAMME.R', 
#     'learning_rate': 0.75, 
#     'n_estimators': 1000, 
#     'random_state': 0},
#   'score': 0.81481481481481477},
#  {'params': {'algorithm': 'SAMME',
#    'learning_rate': 0.75,
#    'n_estimators': 900,
#    'random_state': 0},
#   'score': 0.8204264870931538}]



# gb_bestscore = [{'params': {'max_depth': 10,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 1500,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.81593714927048255}]


# svc_bestscore = [{'params': {'C': 1, 
#     'kernel': 'linear', 
#     'random_state': 0},
#   'score': 0.811447811448}]

In [8]:
# rf_bestscore = [{'params': {'criterion': 'gini',
#    'max_depth': 5,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 200,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0,
#    'warm_start': True},
#   'score': 0.834733893557423},
#  {'params': {'criterion': 'gini',
#    'max_depth': 5,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 190,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0,
#    'warm_start': True},
#   'score': 0.84033613445378152}]

# et_bestscore = [{'params': {'max_depth': 10,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 200,
#    'n_jobs': -1,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.82072829131652658}]

# ada_bestscore = [{'params': {'algorithm': 'SAMME',
#    'learning_rate': 0.75,
#    'n_estimators': 900,
#    'random_state': 0},
#   'score': 0.8123249299719888},
#  {'params': {'algorithm': 'SAMME',
#    'learning_rate': 0.5,
#    'n_estimators': 100,
#    'random_state': 0},
#   'score': 0.81512605042016806},
#  {'params': {'algorithm': 'SAMME',
#    'learning_rate': 0.5,
#    'n_estimators': 130,
#    'random_state': 0},
#   'score': 0.81792717086834732},
#  {'params': {'algorithm': 'SAMME',
#    'learning_rate': 0.45,
#    'n_estimators': 120,
#    'random_state': 0},
#   'score': 0.82072829131652658}]

# gb_bestscore = [{'params': {'max_depth': 20,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 1600,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.80952380952380953},
#  {'params': {'max_depth': 50,
#    'max_features': 'sqrt',
#    'min_samples_leaf': 2,
#    'n_estimators': 2500,
#    'random_state': 0,
#    'verbose': 0},
#   'score': 0.81792717086834732}]

# svc_bestscore = [{'params': {'C': 1, 'kernel': 'linear', 'random_state': 0},
#   'score': 0.82352941176470584}]

In [9]:
rf_bestscore = [{'params': {'criterion': 'gini',
   'max_depth': 5,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'n_estimators': 190,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.85854341736694684},
 {'params': {'criterion': 'gini',
   'max_depth': 10,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 100,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.87394957983193278},
 {'params': {'criterion': 'gini',
   'max_depth': 20,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 200,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.87675070028011204}]

et_bestscore = []

ada_bestscore = []

gb_bestscore = []

svc_bestscore = []

In [14]:
rf_params = {
    'n_estimators': [180, 200, 220], # how many trees?
    'max_depth': [19, 20, 21, 23],
    'n_jobs': [-1],
    'min_samples_leaf': [2],       # minimum samples in a leaf
    'max_features' : ['sqrt', 'log2', None],
    'criterion':['gini'],
    'verbose': [0],                     # don't tell me what happened
     'warm_start': [True]               # progress when generating new trees
}

test_params = gen_params(rf_params)
best_score, best_param = find_best_params(test_params, RandomForestClassifier)

if rf_bestscore == [] or (best_score > sorted(rf_bestscore, key=lambda x:x['score'], reverse=True)[0]['score']):
    rf_bestscore.append({'params':best_param, 'score':best_score})

rf_bestscore

number: 0
params: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 220, 'n_jobs': -1, 'verbose': 0, 'warm_start': True, 'random_state': 0}
accuracy: 0.865546218487
number: 2
params: {'criterion': 'gini', 'max_depth': 23, 'max_features': None, 'min_samples_leaf': 2, 'n_estimators': 200, 'n_jobs': -1, 'verbose': 0, 'warm_start': True, 'random_state': 0}
accuracy: 0.878151260504


[{'params': {'criterion': 'gini',
   'max_depth': 5,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'n_estimators': 190,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.8585434173669468},
 {'params': {'criterion': 'gini',
   'max_depth': 10,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 100,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.8739495798319328},
 {'params': {'criterion': 'gini',
   'max_depth': 20,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 200,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.876750700280112},
 {'params': {'criterion': 'gini',
   'max_depth': 23,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 200,
   'n_jobs': -1,
   'random_state': 0,
   'verbose': 0,
   'warm_start': True},
  'score': 0.87815126050420167}]

In [ ]:
et_params = {
    'n_jobs': [-1],
    'n_estimators':[191, 193, 195, 197, 199],
    'max_features': ['log2', 'sqrt', None],
    'max_depth': [10],
    'min_samples_leaf': [2],
    'verbose': [0]
}

test_params = gen_params(et_params)
best_score, best_param = find_best_params(test_params, ExtraTreesClassifier)

if et_bestscore == [] or (best_score > sorted(et_bestscore, key=lambda x:x['score'], reverse=True)[0]['score']):
    et_bestscore.append({'params':best_param, 'score':best_score})

et_bestscore

In [ ]:
ada_params = {
    'n_estimators': [115, 120, 125],
    'learning_rate' : [0.41, 0.43, 0.45, 0.47, 0.49],
    'algorithm' : ['SAMME']
}

test_params = gen_params(ada_params)

for item in test_params:
    for key, value in item.items():
        if key == 'n_estimators':
            item[key] = int(value)

best_score, best_param = find_best_params(test_params, AdaBoostClassifier)

if ada_bestscore == [] or (best_score > sorted(ada_bestscore, key=lambda x:x['score'], reverse=True)[0]['score']):
    ada_bestscore.append({'params':best_param, 'score':best_score})

ada_bestscore

In [ ]:
gb_params = {
    'n_estimators': [1600, 1800, 2500, 3000, 4000],
    'max_features': ['log2', 'sqrt', None],
    'max_depth': [30, 50, 60],
    'min_samples_leaf': [2],
    'verbose': [0]
}

test_params = gen_params(gb_params)
best_score, best_param = find_best_params(test_params, GradientBoostingClassifier)

if gb_bestscore == [] or (best_score > sorted(gb_bestscore, key=lambda x:x['score'], reverse=True)[0]['score']):
    gb_bestscore.append({'params':best_param, 'score':best_score})

gb_bestscore

In [ ]:
svc_params_Li = [
    {
    'C' : [1,2,3,5,10],
    'kernel' : ['linear', 'rbf', 'sigmoid'],  ## poly takes too much time
    },
#     {
#     'C' : [1,2,3],
#     'kernel' : ['poly'],
#     'degree': [2,3,4,5] # if use poly
#     },
#     {
#     'C' : [1,2,3,5,10],
#     'kernel' : ['rbf', 'sigmoid'],
#     'gamma' : ['auto', 0.2, 0.3, 0.4]
#     }
]


for svc_params in svc_params_Li:
    print('==========================')
    print('new svc_params')
    test_params = gen_params(svc_params)

    for item in test_params:
        for key, value in item.items():
            if key == 'kernel':
                item[key] = str(value)
            if key == 'C':
                item[key] = int(value)
            if key == 'degree':
                item[key] = int(value)
            if key == 'gamma':
                if value != 'auto':
                    item[key] = float(value)
                
    best_score, best_param = find_best_params(test_params, SVC)

    if svc_bestscore == [] or (best_score > sorted(svc_bestscore, key=lambda x:x['score'], reverse=True)[0]['score']):
        svc_bestscore.append({'params':best_param, 'score':best_score})

    from pprint import pprint
    pprint(svc_bestscore)

In [ ]:
# 三次tuning random forest的比較

In [ ]:
rf_params1 = {'criterion': 'gini',
   'max_depth': 25,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 345,
   'n_jobs': -1,
   'verbose': 0,
   'warm_start': True}
# 0.75598

rf_params2 = {'criterion': 'gini',
   'max_depth': 5,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'n_estimators': 190,
   'n_jobs': -1,
   'verbose': 0,
   'warm_start': True}
# 0.78468

rf_params3 = {'criterion': 'gini',
   'max_depth': 20,
   'max_features': None,
   'min_samples_leaf': 2,
   'n_estimators': 200,
   'n_jobs': -1,
   'verbose': 0,
   'warm_start': True}
# 0.76555

rf_params4 = {'criterion': 'gini',
   'max_depth': 4,
   'max_features': 'sqrt',
   'min_samples_leaf': 2,
   'n_estimators': 180,
   'n_jobs': -1,
   'verbose': 0,
   'warm_start': True}


# rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params1)
# rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params2)
# rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params3)
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params4)

rf_oof_train, rf_oof_test, rf_accuracy = get_oof(rf,x_train, y_train, x_test) # Random Forest
# Generate Submission File 
predictions = rf_oof_test.reshape(len(rf_oof_test), ).astype(int)

StackingSubmission = pd.DataFrame({ 'PassengerId': ID, 'Survived': predictions })

nowtime = str(datetime.now())
for char in string.punctuation:
    nowtime = nowtime.replace(char, "").replace(" ", "")

# filename = os.path.join("submission", nowtime[:14]+"rf_params1.csv")
# filename = os.path.join("submission", nowtime[:14]+"rf_params2.csv")
# filename = os.path.join("submission", nowtime[:14]+"rf_params3.csv")
filename = os.path.join("submission", nowtime[:14]+"rf_params4.csv")

StackingSubmission.to_csv(filename, index=False)